In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_train = pd.read_csv('../input/background-removed-happywhale-dataset/seg_train.csv')
df_train

In [ ]:
df_train.describe()

In [ ]:
df_corr = df_train.corr
df_corr

In [ ]:
df_train['box'] = df_train['box'].fillna(0)
df_train

In [ ]:
df_train["image"].replace('.png')
df_train["image"]

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
DATA_DIR  = '../input/happy-whale-and-dolphin/'
TRAIN_DIR = DATA_DIR + 'train_images/'
# TEST_DIR  = DATA_DIR + 'resized_test_images/'



label_encoder = LabelEncoder()

# Load Train Data
train_df = pd.read_csv('../input/background-removed-happywhale-dataset/seg_train.csv')
train_df['Id'] = train_df['image'].apply(lambda x: f'{TRAIN_DIR}{x}')

# Adjust typos in "species" column from Andrada's kernel
train_df["species"] = train_df["species"].replace(["bottlenose_dolpin", "kiler_whale",
                                             "beluga", 
                                             "globis", "pilot_whale"],
                                            ["bottlenose_dolphin", "killer_whale",
                                             "beluga_whale", 
                                             "short_finned_pilot_whale", "short_finned_pilot_whale"])


# Set a specific label to be able to perform stratification
#train_df['stratify_label'] = train_df['individual_id']

train_df['target_value']  = label_encoder.fit_transform(train_df['individual_id'] )

# Summary
print(f'train_df: {train_df.shape}')
train_df.head()


In [ ]:
train_df['box'] = train_df['box'].astype("string")
train_df.to_csv('train.csv')

In [ ]:
train_df.info

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [ ]:
train_df["box"] = train_df["box"].fillna(0)
train_df["box"] = train_df["box"].astype("string")

In [ ]:
train_df["box"] = train_df["box"].fillna(0)

In [ ]:

datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    preprocessing_function=preprocess_input,
    validation_split=0.3)


train_generator=datagen.flow_from_dataframe(
dataframe=train_df,
# directory="../input/happy-whale-and-dolphin/train_images",
x_col="Id",
y_col=["individual_id","box"],
subset="training",
batch_size=256,
# seed=2022,
shuffle=True,
class_mode="multi_output",
target_size=(512,512))

valid_generator=datagen.flow_from_dataframe(
dataframe=train_df,
# directory="../input/happy-whale-and-dolphin/train_images",
x_col="Id",
y_col=["individual_id","box"],
subset="validation",
batch_size=256,
# seed=2022,
shuffle=True,
class_mode="multi_output",
target_size=(512,512))

In [ ]:
import cv2
img = cv2.imread('../input/background-removed-happywhale-dataset/seg_img/00021adfb725ed.png')
img.shape

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(512,512, 3))

# Construct the head of the model that will be placed on top of the base model
head_model = base_model.output
head_model = AveragePooling2D(pool_size=(3,3))(head_model)
head_model = Flatten(name="flatten")(head_model)
# head_model = Dropout(0.5)(head_model)
head_model = Dense(512, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(15587, activation="softmax")(head_model)

# Place the head FC model on top of the base model (this will become the actual model we will train)
model = Model(inputs=base_model.input, outputs=head_model)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
model.summary()

In [ ]:
opt = Adam(learning_rate=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
H = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
#     callbacks=[early_stopping],
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size,
    epochs=4)
